# 2. Implementation

## 2.1 State Space Search

### Helper methods

In [61]:
import copy

def isGoalState(puzzle):
    if puzzle[2][4] == "A" and puzzle[2][5] == "A":
        return True
    return False

def runValetServiceCheckup(newPuzzle):
    if newPuzzle[2][5] != "A" and newPuzzle[2][5] == newPuzzle[2][4]:
        #car is horizontal and is ready for valet
        carType = newPuzzle[2][5]
        newCol = 5
        while(newCol>=0):
            if newPuzzle[2][newCol] == carType:
                newPuzzle[2][newCol] = "."
            else:
                break
            newCol -= 1
    return newPuzzle

def findHorizontalMovesOfCar(solutionPath, puzzle, y, startX, endX, fuelLevels):
    newNodes = []
    carLength = endX - startX + 1
    carType = puzzle[y][startX]
    for posX in reversed(range(startX)):
        #move car left
        if puzzle[y][posX] != ".":
            break

        newPuzzle = copy.deepcopy(puzzle)
        newFuelLevels = copy.deepcopy(fuelLevels)
        newSolutionPath = copy.deepcopy(solutionPath)

        displacement = startX - posX
        if displacement <= newFuelLevels[carType]:
            newFuelLevels[carType] = newFuelLevels[carType] - displacement
            newSolutionPath.append({
                "carType":carType,
                "direction":"left",
                "displacement":displacement
            })

            carPartsToMove = carLength
            for newIndexX in range(posX, endX+1):
                if carPartsToMove > 0:
                    newPuzzle[y][newIndexX] = carType
                    carPartsToMove -= 1
                else:
                    newPuzzle[y][newIndexX] = "."

            newPuzzle = runValetServiceCheckup(newPuzzle)
            newNode = {
                "puzzle":newPuzzle,
                "solutionPath":newSolutionPath,
                "fuelLevels":newFuelLevels
            }
            newNodes.append(newNode)
        else:
            break
    for posX in range(endX+1, 6):
        #move car right
        if puzzle[y][posX] != ".":
            break

        newPuzzle = copy.deepcopy(puzzle)
        newFuelLevels = copy.deepcopy(fuelLevels)
        newSolutionPath = copy.deepcopy(solutionPath)

        displacement = posX - endX
        if displacement <= fuelLevels[carType]:
            newFuelLevels[carType] = newFuelLevels[carType] - displacement
            newSolutionPath.append({
                "carType":carType,
                "direction":"right",
                "displacement":displacement
            })

            carPartsToMove = carLength
            for newIndexX in reversed(range(startX, posX+1)):
                if carPartsToMove > 0:
                    newPuzzle[y][newIndexX] = carType
                    carPartsToMove -= 1
                else:
                    newPuzzle[y][newIndexX] = "."

            newPuzzle = runValetServiceCheckup(newPuzzle)
            newNode = {
                "puzzle":newPuzzle,
                "solutionPath":newSolutionPath,
                "fuelLevels":newFuelLevels
            }
            newNodes.append(newNode)
        else:
            break
    return newNodes

def findVerticalMovesOfCar(solutionPath, puzzle, x, startY, endY, fuelLevels):
    newNodes = []
    carLength = endY - startY + 1
    carType = puzzle[startY][x]
    for posY in reversed(range(startY)):
        #move car up
        if puzzle[posY][x] != ".":
            break

        newPuzzle = copy.deepcopy(puzzle)
        newFuelLevels = copy.deepcopy(fuelLevels)
        newSolutionPath = copy.deepcopy(solutionPath)

        displacement = startY - posY
        if displacement <= fuelLevels[carType]:
            newFuelLevels[carType] = newFuelLevels[carType] - displacement
            newSolutionPath.append({
                "carType":carType,
                "direction":"up",
                "displacement":displacement
            })

            carPartsToMove = carLength
            for newIndexY in range(posY, endY+1):
                if carPartsToMove > 0:
                    newPuzzle[newIndexY][x] = carType
                    carPartsToMove -= 1
                else:
                    newPuzzle[newIndexY][x] = "."

            newPuzzle = runValetServiceCheckup(newPuzzle)
            newNode = {
                "puzzle":newPuzzle,
                "solutionPath":newSolutionPath,
                "fuelLevels":newFuelLevels
            }
            newNodes.append(newNode)
        else:
            break
    for posY in range(endY+1, 6):
        #move car down
        if puzzle[posY][x] != ".":
            break

        newPuzzle = copy.deepcopy(puzzle)
        newFuelLevels = copy.deepcopy(fuelLevels)
        newSolutionPath = copy.deepcopy(solutionPath)

        displacement = posY - endY
        if displacement <= fuelLevels[carType]:
            newFuelLevels[carType] = newFuelLevels[carType] - displacement
            newSolutionPath.append({
                "carType":carType,
                "direction":"down",
                "displacement":displacement
            })

            carPartsToMove = carLength
            for newIndexY in reversed(range(startY, posY+1)):
                if carPartsToMove > 0:
                    newPuzzle[newIndexY][x] = carType
                    carPartsToMove -= 1
                else:
                    newPuzzle[newIndexY][x] = "."

            newPuzzle = runValetServiceCheckup(newPuzzle)
            newNode = {
                "puzzle":newPuzzle,
                "solutionPath":newSolutionPath,
                "fuelLevels":newFuelLevels
            }
            newNodes.append(newNode)
        else:
            break
    return newNodes

def findSuccessors(node):
    newNode = copy.deepcopy(node)
    solutionPath = newNode["solutionPath"]
    successorsList = []
    puzzle = newNode["puzzle"]
    movedCars = []
    fuelLevels = newNode["fuelLevels"]
    for row in range(len(puzzle)):
        for col in range(len(puzzle[row])):
            if (puzzle[row][col] != ".") and (puzzle[row][col] not in movedCars):
                movedCars.append(puzzle[row][col])
                if (col+1 != 6) and (puzzle[row][col] == puzzle[row][col+1]):
                    #horizontal car
                    endX = col
                    for newX in range(col, len(puzzle[row])):
                        if puzzle[row][newX] != puzzle[row][col]:
                            break
                        else:
                            endX = newX
                    horizontalMoves = findHorizontalMovesOfCar(solutionPath, puzzle, row, col, endX, fuelLevels)
                    successorsList = successorsList + horizontalMoves
                elif (row+1 != 6) and (puzzle[row][col] == puzzle[row+1][col]):
                    #vertical car
                    endY = row
                    for newY in range(row, len(puzzle)):
                        if puzzle[newY][col] != puzzle[row][col]:
                            break
                        else:
                            endY = newY
                    verticalMoves = findVerticalMovesOfCar(solutionPath, puzzle, col, row, endY, fuelLevels)
                    successorsList = successorsList + verticalMoves
    return successorsList

def arePuzzlesEqual(puzzle1, puzzle2):
    for row in range(len(puzzle1)):
        for col in range(len(puzzle1[row])):
            if puzzle1[row][col] != puzzle2[row][col]:
                return False
    return True

def printCutePuzzle(puzzle):
    for line in puzzle:
        str = ""
        for elem in line:
            str += elem + " "
        print(str+"\n")

def solver(puzzle, fuelLevels, heuristic):
    initialState = {
        "puzzle":puzzle,
        "solutionPath":[],
        "fuelLevels":fuelLevels
    }
    openList = [initialState]
    closedList = []

    nbSimilar = 0
    nbSimilar2 = 0

    while(len(openList) > 0):
        firstNode = openList.pop(0)
        if isGoalState(firstNode["puzzle"]):
            printCutePuzzle(firstNode["puzzle"])
            print(firstNode["solutionPath"])
            return firstNode["solutionPath"]
        else:
            closedList.append(firstNode)
            successors = findSuccessors(firstNode)
            for successor in successors:
                isAlreadyExplored = False
                for closedNode in closedList:
                    if arePuzzlesEqual(successor["puzzle"], closedNode["puzzle"]):
                        isAlreadyExplored = True
                        nbSimilar+=1
                        break
                if not isAlreadyExplored:
                    isAlreadyInOpenList = False
                    for i in range(len(openList)):
                        if arePuzzlesEqual(successor["puzzle"], openList[i]["puzzle"]):
                            isAlreadyInOpenList = True
                            nbSimilar2 += 1
                            if len(successor["solutionPath"]) < len(openList[i]["solutionPath"]):
                                openList[i] = successor
                    if not isAlreadyInOpenList:
                        openList.append(successor)
            openList.sort(key=heuristic, reverse=False)

        print("\rNb similar (closed): "+str(nbSimilar)+"; Nb similar (open): "+str(nbSimilar2)+"; Len closedList: "+str(len(closedList))+"; Len open list: "+str(len(openList)), end="");

    print("NO SOLUTION!")
    return None

#### heuristics

In [62]:
def h1(node):
    puzzle = node["puzzle"]
    isAmbulanceFound = False
    uniqueCarTypes = []
    for col in range(0, 6):
        value = puzzle[2][col]
        if isAmbulanceFound == True:
            if (value != "A") and (value != ".") and (value not in uniqueCarTypes):
                uniqueCarTypes.append(value)
        else:
            if value == "A":
                isAmbulanceFound = True
    return len(uniqueCarTypes)

def h2(node):
    puzzle = node["puzzle"]
    isAmbulanceFound = False
    count = 0
    for col in range(0, 6):
        value = puzzle[2][col]
        if isAmbulanceFound == True:
            if (value != "A") and (value != "."):
                count += 1
        else:
            if value == "A":
                isAmbulanceFound = True
    return count

def h3(node):
    return h1(node) * 3

def h4(node):
    #like h1, but count car twice if it is itself blocked
    puzzle = node["puzzle"]
    isAmbulanceFound = False
    uniqueCarTypes = []
    count = 0
    for col in range(0, 6):
        value = puzzle[2][col]
        if isAmbulanceFound == True:
            #search for cars to the right of ambulance
            if (value != "A") and (value != ".") and (value not in uniqueCarTypes):
                uniqueCarTypes.append(value)
                count += 1
                if col < 5 and puzzle[2][col+1] == value:
                    #car is horizontal
                    if puzzle[2][col-1] != ".":
                        #is blocked left
                        newCol = col
                        isBlocked = False
                        while(newCol <= 5):
                            if puzzle[2][newCol] == ".":
                                break
                            elif puzzle[2][newCol] != value:
                                isBlocked = True
                                #is also blocked right
                                break
                            newCol+=1
                        if isBlocked:
                            count+=1
                elif puzzle[1][col] == value or puzzle[3][col] == value:
                    #car is vertical
                    newRowUp = 1
                    isBlockedUp = False
                    while(newRowUp >= 0):
                        if puzzle[newRowUp][col] == ".":
                            break
                        elif puzzle[newRowUp][col] != value:
                            isBlockedUp = True
                            #is blocked up
                            break
                        newRowUp -= 1
                    if isBlockedUp:
                        newRowDown = 3
                        isBlockedDown = False
                        while(newRowDown<=5):
                            if puzzle[newRowDown][col] == ".":
                                break
                            elif puzzle[newRowDown][col] != value:
                                isBlockedDown = True
                                #is blocked down
                                break
                            newRowDown += 1
                        if isBlockedDown:
                            count+=1

        else:
            if value == "A":
                isAmbulanceFound = True
    return count

### 1. Uniform Cost Search

In [63]:
def solveUCS(puzzle, fuelLevels):
    heuristic = lambda x:len(x["solutionPath"])
    solver(puzzle, fuelLevels, heuristic)

### 2. Greedy Best First Search

In [64]:
def solveGBFS(puzzle, fuelLevels):
    print("Heuristic 1")
    solver(puzzle, fuelLevels, h1)

    print("Heuristic 2")
    solver(puzzle, fuelLevels, h2)

    print("Heuristic 3")
    solver(puzzle, fuelLevels, h3)

    print("Heuristic 4")
    solver(puzzle, fuelLevels, h4)
    return 2

### 3. Algorithm A or A*

#### Algorithm A heuristics

In [65]:
def h1A(node):
    return len(node["solutionPath"]) + h1(node)

def h2A(node):
    return len(node["solutionPath"]) + h2(node)

def h3A(node):
    return len(node["solutionPath"]) + h3(node)

def h4A(node):
    return len(node["solutionPath"]) + h4(node)

In [66]:
def solveA(puzzle, fuelLevels):
    print("Heuristic 1")
    solver(puzzle, fuelLevels, h1A)

    print("Heuristic 2")
    solver(puzzle, fuelLevels, h2A)

    print("Heuristic 3")
    solver(puzzle, fuelLevels, h3A)

    print("Heuristic 4")
    solver(puzzle, fuelLevels, h4A)


## 2.2 Input

In [67]:
def readInputSamples():
    input_file = open('./Sample/sample-input.txt', 'r')
    lines = input_file.readlines()
    is_next_line_puzzle = False
    count = 0
    for line in lines:
        fuel_levels = {}
        puzzle = []
        if is_next_line_puzzle:
            line = line.strip();
            splitted_line = line.split(" ")
            puzzle_string = splitted_line[0]

            puzzle_row = []
            for i in range(len(puzzle_string)):
                if puzzle_string[i] not in fuel_levels:
                    fuel_levels[puzzle_string[i]] = 100
                puzzle_row.append(puzzle_string[i])
                if len(puzzle_row) == 6:
                    puzzle.append(puzzle_row)
                    puzzle_row = []

            fuel_levels_array = splitted_line[1:]
            for fuel_level in fuel_levels_array:
                fuel_levels[fuel_level[0]] = int(fuel_level[1])
            is_next_line_puzzle = False
            if count == 5:
               printCutePuzzle(puzzle)
               solveA(puzzle, fuel_levels)
            count+=1
        elif line.__contains__("demo"):
            print(line.strip())
            is_next_line_puzzle = True

    input_file.close()

readInputSamples()

# demo 1.1
# demo 1.2
# demo 1.3
# demo 1.4
# demo 1.5
# demo 1.6
B B . G . H 

E . . G . H 

E A A G . I 

. . F C C I 

D D F . . I 

. . F . . . 

Heuristic 1
Nb similar (closed): 2126; Nb similar (open): 1625; Len closedList: 646; Len open list: 147E . . B B H 

E . . . . H 

. . . . A A 

C C F G . I 

D D F G . I 

. . F G . I 

[{'carType': 'I', 'direction': 'down', 'displacement': 1}, {'carType': 'B', 'direction': 'right', 'displacement': 1}, {'carType': 'E', 'direction': 'up', 'displacement': 1}, {'carType': 'A', 'direction': 'left', 'displacement': 1}, {'carType': 'F', 'direction': 'up', 'displacement': 2}, {'carType': 'D', 'direction': 'right', 'displacement': 3}, {'carType': 'F', 'direction': 'down', 'displacement': 2}, {'carType': 'A', 'direction': 'right', 'displacement': 1}, {'carType': 'E', 'direction': 'down', 'displacement': 4}, {'carType': 'B', 'direction': 'left', 'displacement': 1}, {'carType': 'A', 'direction': 'left', 'displacement': 1}, {'carType': 'F', 'directi

## 2.3 Output

### 2.3.1 Solution Files

### 2.3.2 Search Files

# 3. Analysis